# Working with Time Series

Pandas was developed in the context of financial modeling, so as you might expect, it contains a fairly extensive set of tools for working with dates, times, and time-indexed data. Date and time data comes in a few flavors, which we will discuss here:

- *Time stamps* reference particular moments in time (e.g, July 4th, 2015 at 7:00 am).
- *Time intervals* and *periods* reference a length of time between a particular beginning and en point; for example, the year 2015. Periods usually reference a special case of time intervals in which each interval is of uniform length and does not overlap(e.g, 24 hour-long periods comprising days).
- *Time deltas* or *durations* reference an exact length of time (e.g., duration of 22.56 seconds).

In this section, we will introduce how to work with each of these types of date/time data in Pandas. This short sectio nis by no means a complete guide to the time series tools available in Python or Pandas, but instead is intended as a broad overview of how you as a user should approach working with time series. We will star twith a brief discussion of tools for dealing with dates and times in Python, before moving more specifically to a discussion of the tools provided by Pandas. After listing some resources that go into more depth, we will review some short examples of working with time series data in Pandas.